In [1]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import wandb
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import r2_score
from matplotlib.colors import ListedColormap, Normalize
from matplotlib.cm import ScalarMappable
from matplotlib.lines import Line2D
from datetime import datetime
from sklearn.preprocessing import StandardScaler
import os
from helper import train_VAE 

In [2]:
X_train = pd.read_csv('../datasets/Bikerental/X_train.csv',index_col=0)  
X_test = pd.read_csv('../datasets/Bikerental/X_test.csv',index_col=0)  
y_train = pd.read_csv('../datasets/Bikerental/y_train.csv',index_col=0)  
y_test = pd.read_csv('../datasets/Bikerental/y_test.csv',index_col=0) 
X_train_with_confounder=X_train.drop(columns=['season_SPRING','season_FALL','season_SUMMER','season_WINTER'])
X_test_with_confounder=X_test.drop(columns=['season_SPRING','season_FALL','season_SUMMER','season_WINTER'])


In [3]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_with_confounder)
X_test_scaled = scaler.transform(X_test_with_confounder)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
def custom_scale(data, min_val=0, max_val=10000):
    scaled_data = (data - min_val) / (max_val - min_val)
    return scaled_data

# Apply custom scaling to y_train and y_test
y_train_scaled = custom_scale(y_train)
y_test_scaled = custom_scale(y_test)

# Parameter space

In [5]:
import itertools

# Define the ranges for your parameters
x_dim_values = [X_train_with_confounder.shape[1]]
z_dim_values = [4]
zz_dim_values = [1]
encoder_z_layers_values = [[12,12,12],[128,128]]
encoder_zz_layers_values = [[5,5,5]]
fc_z_to_y_layers_values = [[28,28,1]]

# Get all combinations
configurations = list(itertools.product(x_dim_values, z_dim_values, zz_dim_values, encoder_z_layers_values, encoder_zz_layers_values, fc_z_to_y_layers_values))

# Convert to a list of dictionaries
config_dicts = [{'x_dim': x_dim, 'z_dim': z_dim, 'zz_dim': zz_dim, 'encoder_z_layers': encoder_z_layers, 'encoder_zz_layers': encoder_zz_layers, 'decoder_layers': encoder_z_layers+ [x_dim], 'fc_z_to_y_layers': fc_z_to_y_layers} for x_dim, z_dim, zz_dim, encoder_z_layers, encoder_zz_layers, fc_z_to_y_layers in configurations]
print(config_dicts)

[{'x_dim': 10, 'z_dim': 4, 'zz_dim': 1, 'encoder_z_layers': [12, 12, 12], 'encoder_zz_layers': [5, 5, 5], 'decoder_layers': [12, 12, 12, 10], 'fc_z_to_y_layers': [28, 28, 1]}, {'x_dim': 10, 'z_dim': 4, 'zz_dim': 1, 'encoder_z_layers': [128, 128], 'encoder_zz_layers': [5, 5, 5], 'decoder_layers': [128, 128, 10], 'fc_z_to_y_layers': [28, 28, 1]}]


In [6]:
# Define betas
BETAS=[
    #{'b1': 6, 'b2': 1, 'b3': 1, 'b4': 6,'b5':1},
    #{'b1': 1, 'b2': 1, 'b3': 1, 'b4': 1000,'b5':1},
    #{'b1': 100, 'b2': 1, 'b3': 100, 'b4': 100,'b5':1},
    #{'b1': 100,'b2': 1, 'b3': 1, 'b4': 10,'b5':1},  
    #{'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 10000,'b5':1},
    {'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 1000,'b5':1}
]
import itertools

# Define the ranges for your parameters
lr_values = [1e-3]
batch_size_values = [16,32]
num_epochs_values = [700,1000]

# Get all combinations
configs = list(itertools.product(config_dicts, BETAS, lr_values, batch_size_values, num_epochs_values))

# Convert to a list of dictionaries
configs = [{'config': config, 'beta': beta, 'lr': lr, 'batch_size': bs, 'num_epochs': epochs} for config, beta, lr, bs, epochs in configs]
print(len(configs))

8


# No Freeze and z and zz to y 

In [8]:
from sklearn.model_selection import KFold
from tqdm import tqdm
kf = KFold(n_splits=5, shuffle=True)
results = pd.DataFrame()

In [11]:
df_list = []
for conf in tqdm(configs):
    r2s=[]
    r2s_test=[]
    losses_train=[]
    losses_test=[]
    for train_index, test_index in kf.split(X_train_scaled):
        X_train_cv, X_val_cv = X_train_scaled[train_index], X_train_scaled[test_index]
        y_train_cv, y_val_cv = y_train_scaled.to_numpy()[train_index], y_train_scaled.to_numpy()[test_index]
        #y_train_cv, y_val_cv = y_train.to_numpy()[train_index], y_train.to_numpy()[test_index]
        model,r2,loss_train,loss_test=train_VAE(X_train_cv,X_val_cv,X_val_cv,y_train_cv,y_val_cv,conf,'./Test',save_pictures=False,freeze_weights=False)
        r2s.append(r2)
        losses_train.append(loss_train)
        losses_test.append(loss_test)
        test_ds = TensorDataset(torch.Tensor(X_test_scaled), torch.Tensor(y_test_scaled.values))
        y_true = np.array([y.numpy() for _, y in test_ds],dtype=object)
        _,_,_,_,_,outputs = model(torch.Tensor(X_test_scaled).to(device))
        y_pred = outputs.cpu().detach().numpy()
        r2_test = r2_score(y_true, y_pred)
        r2s_test.append(r2_test)
        print("R^2 Test score:", r2_test)
    # Adding each config, mean R2 score, mean loss_train, and mean loss_test to the dictionary
    df_list.append({'config': conf, 
                    'mean_r2': np.mean(r2s),
                    'mean_r2_test': np.mean(r2s_test),
                    'mean_loss_train': np.mean(losses_train),
                    'mean_loss_test': np.mean(losses_test)})

# Creating DataFrame from list of dictionaries
df = pd.DataFrame(df_list)


  0%|          | 0/8 [00:00<?, ?it/s]

{'config': {'x_dim': 10, 'z_dim': 4, 'zz_dim': 1, 'encoder_z_layers': [12, 12, 12], 'encoder_zz_layers': [5, 5, 5], 'decoder_layers': [12, 12, 12, 10], 'fc_z_to_y_layers': [28, 28, 1]}, 'beta': {'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 1000, 'b5': 1}, 'lr': 0.001, 'batch_size': 16, 'num_epochs': 700}
False
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 12]             132
              ReLU-2                   [-1, 12]               0
            Linear-3                   [-1, 12]             156
              ReLU-4                   [-1, 12]               0
            Linear-5                   [-1, 12]             156
              ReLU-6                   [-1, 12]               0
            Linear-7                    [-1, 4]              52
            Linear-8                    [-1, 4]              52
            Linear-9                    [-1, 5]        

Epoch 1, loss: 9046.9169921875
Epoch 201, loss: 2585.42333984375
Epoch 401, loss: 365.2866516113281
Epoch 601, loss: 741.3298950195312
R^2 score: 0.7608717556664497
R^2 Test score: 0.7300914876404675
{'config': {'x_dim': 10, 'z_dim': 4, 'zz_dim': 1, 'encoder_z_layers': [12, 12, 12], 'encoder_zz_layers': [5, 5, 5], 'decoder_layers': [12, 12, 12, 10], 'fc_z_to_y_layers': [28, 28, 1]}, 'beta': {'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 1000, 'b5': 1}, 'lr': 0.001, 'batch_size': 16, 'num_epochs': 700}
False
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 12]             132
              ReLU-2                   [-1, 12]               0
            Linear-3                   [-1, 12]             156
              ReLU-4                   [-1, 12]               0
            Linear-5                   [-1, 12]             156
              ReLU-6                   [-1, 12]

 12%|█▎        | 1/8 [12:09<1:25:05, 729.34s/it]

R^2 score: 0.7620806559996929
R^2 Test score: 0.7713139583949626
{'config': {'x_dim': 10, 'z_dim': 4, 'zz_dim': 1, 'encoder_z_layers': [12, 12, 12], 'encoder_zz_layers': [5, 5, 5], 'decoder_layers': [12, 12, 12, 10], 'fc_z_to_y_layers': [28, 28, 1]}, 'beta': {'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 1000, 'b5': 1}, 'lr': 0.001, 'batch_size': 16, 'num_epochs': 1000}
False
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 12]             132
              ReLU-2                   [-1, 12]               0
            Linear-3                   [-1, 12]             156
              ReLU-4                   [-1, 12]               0
            Linear-5                   [-1, 12]             156
              ReLU-6                   [-1, 12]               0
            Linear-7                    [-1, 4]              52
            Linear-8                    [-1, 4]      

...

 88%|████████▊ | 7/8 [1:16:51<10:03, 603.95s/it]

R^2 score: 0.8500526344047151
R^2 Test score: 0.8216331335291639
{'config': {'x_dim': 10, 'z_dim': 4, 'zz_dim': 1, 'encoder_z_layers': [128, 128], 'encoder_zz_layers': [5, 5, 5], 'decoder_layers': [128, 128, 10], 'fc_z_to_y_layers': [28, 28, 1]}, 'beta': {'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 1000, 'b5': 1}, 'lr': 0.001, 'batch_size': 32, 'num_epochs': 1000}
False
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                  [-1, 128]           1,408
              ReLU-2                  [-1, 128]               0
            Linear-3                  [-1, 128]          16,512
              ReLU-4                  [-1, 128]               0
            Linear-5                    [-1, 4]             516
            Linear-6                    [-1, 4]             516
            Linear-7                    [-1, 5]              25
              ReLU-8                    [-1, 5]          

Epoch 201, loss: 390.9759216308594
Epoch 401, loss: 282.6277770996094
Epoch 601, loss: 331.42730712890625
Epoch 801, loss: 292.1014404296875
R^2 score: 0.8036616485896709
R^2 Test score: 0.8228908541606577
{'config': {'x_dim': 10, 'z_dim': 4, 'zz_dim': 1, 'encoder_z_layers': [128, 128], 'encoder_zz_layers': [5, 5, 5], 'decoder_layers': [128, 128, 10], 'fc_z_to_y_layers': [28, 28, 1]}, 'beta': {'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 1000, 'b5': 1}, 'lr': 0.001, 'batch_size': 32, 'num_epochs': 1000}
False
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                  [-1, 128]           1,408
              ReLU-2                  [-1, 128]               0
            Linear-3                  [-1, 128]          16,512
              ReLU-4                  [-1, 128]               0
            Linear-5                    [-1, 4]             516
            Linear-6                    [-1,

100%|██████████| 8/8 [1:24:46<00:00, 635.77s/it]

R^2 score: 0.8018517038340426
R^2 Test score: 0.8340712871783118


In [12]:
df

,config,mean_r2,mean_r2_test,mean_loss_train,mean_loss_test
0,"{'config': {'x_dim': 10, 'z_dim': 4, 'zz_dim':...",0.735934,0.769397,0.062937,0.097986
1,"{'config': {'x_dim': 10, 'z_dim': 4, 'zz_dim':...",0.780952,0.796643,0.057863,0.090837
2,"{'config': {'x_dim': 10, 'z_dim': 4, 'zz_dim':...",0.769200,0.778629,0.070888,0.091098
3,"{'config': {'x_dim': 10, 'z_dim': 4, 'zz_dim':...",0.765489,0.764791,0.068253,0.093468
4,"{'config': {'x_dim': 10, 'z_dim': 4, 'zz_dim':...",0.802368,0.832992,0.035985,0.084967
5,"{'config': {'x_dim': 10, 'z_dim': 4, 'zz_dim':...",0.776926,0.821782,0.032776,0.090090
6,"{'config': {'x_dim': 10, 'z_dim': 4, 'zz_dim':...",0.810611,0.824108,0.041197,0.082647
7,"{'config': {'x_dim': 10, 'z_dim': 4, 'zz_dim':...",0.777723,0.823038,0.040336,0.090114


In [13]:
df.iloc[6]

config             {'config': {'x_dim': 10, 'z_dim': 4, 'zz_dim':...
mean_r2                                                     0.810611
mean_r2_test                                                0.824108
mean_loss_train                                             0.041197
mean_loss_test                                              0.082647
Name: 6, dtype: object

In [14]:
df.iloc[6].config

{'config': {'x_dim': 10,
  'z_dim': 4,
  'zz_dim': 1,
  'encoder_z_layers': [128, 128],
  'encoder_zz_layers': [5, 5, 5],
  'decoder_layers': [128, 128, 10],
  'fc_z_to_y_layers': [28, 28, 1]},
 'beta': {'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 1000, 'b5': 1},
 'lr': 0.001,
 'batch_size': 32,
 'num_epochs': 700}

# with weight freezing

In [12]:
from sklearn.model_selection import KFold
from tqdm import tqdm
kf = KFold(n_splits=5, shuffle=True)
results = pd.DataFrame()
df_list = []
for conf in tqdm(configs):
    r2s=[]
    r2s_test=[]
    losses_train=[]
    losses_test=[]
    for train_index, test_index in kf.split(X_train_scaled):
        X_train_cv, X_val_cv = X_train_scaled[train_index], X_train_scaled[test_index]
        y_train_cv, y_val_cv = y_train_scaled.to_numpy()[train_index], y_train_scaled.to_numpy()[test_index]
        model,r2,loss_train,loss_test=train_VAE(X_train_cv,X_val_cv,X_val_cv,y_train_cv,y_val_cv,conf,'./Test',save_pictures=False,freeze_weights=True,z_and_zz_to_y=True,offset=500,partial=200)
        r2s.append(r2)
        losses_train.append(loss_train)
        losses_test.append(loss_test)
        test_ds = TensorDataset(torch.Tensor(X_test_scaled), torch.Tensor(y_test_scaled.values))
        y_true = np.array([y.numpy() for _, y in test_ds],dtype=object)
        _,_,_,_,_,outputs = model(torch.Tensor(X_test_scaled).to(device))
        y_pred = outputs.cpu().detach().numpy()
        r2_test = r2_score(y_true, y_pred)
        r2s_test.append(r2_test)
        print("R^2 Test score:", r2_test)
    # Adding each config, mean R2 score, mean loss_train, and mean loss_test to the dictionary
    df_list.append({'config': conf, 
                    'mean_r2': np.mean(r2s),
                    'mean_r2_test': np.mean(r2s_test),
                    'mean_loss_train': np.mean(losses_train),
                    'mean_loss_test': np.mean(losses_test)})

# Creating DataFrame from list of dictionaries
df3 = pd.DataFrame(df_list)


  0%|          | 0/8 [00:00<?, ?it/s]

{'config': {'x_dim': 10, 'z_dim': 4, 'zz_dim': 1, 'encoder_z_layers': [12, 12, 12], 'encoder_zz_layers': [5, 5, 5], 'decoder_layers': [12, 12, 12, 10], 'fc_z_to_y_layers': [28, 28, 1]}, 'beta': {'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 1000, 'b5': 1}, 'lr': 0.001, 'batch_size': 16, 'num_epochs': 700}
False
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 12]             132
              ReLU-2                   [-1, 12]               0
            Linear-3                   [-1, 12]             156
              ReLU-4                   [-1, 12]               0
            Linear-5                   [-1, 12]             156
              ReLU-6                   [-1, 12]               0
            Linear-7                    [-1, 4]              52
            Linear-8                    [-1, 4]              52
            Linear-9                    [-1, 5]        

Epoch 201, loss: 291.5384521484375
Epoch 401, loss: 87.16642761230469
Epoch 601, loss: 46.34100341796875
R^2 score: 0.7151701982832923
R^2 Test score: 0.8006439115487838
{'config': {'x_dim': 10, 'z_dim': 4, 'zz_dim': 1, 'encoder_z_layers': [12, 12, 12], 'encoder_zz_layers': [5, 5, 5], 'decoder_layers': [12, 12, 12, 10], 'fc_z_to_y_layers': [28, 28, 1]}, 'beta': {'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 1000, 'b5': 1}, 'lr': 0.001, 'batch_size': 16, 'num_epochs': 700}
False
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 12]             132
              ReLU-2                   [-1, 12]               0
            Linear-3                   [-1, 12]             156
              ReLU-4                   [-1, 12]               0
            Linear-5                   [-1, 12]             156
              ReLU-6                   [-1, 12]               0
            L

 12%|█▎        | 1/8 [08:52<1:02:07, 532.47s/it]

R^2 score: 0.7121802081393758
R^2 Test score: 0.703594229454747
{'config': {'x_dim': 10, 'z_dim': 4, 'zz_dim': 1, 'encoder_z_layers': [12, 12, 12], 'encoder_zz_layers': [5, 5, 5], 'decoder_layers': [12, 12, 12, 10], 'fc_z_to_y_layers': [28, 28, 1]}, 'beta': {'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 1000, 'b5': 1}, 'lr': 0.001, 'batch_size': 16, 'num_epochs': 1000}
False
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 12]             132
              ReLU-2                   [-1, 12]               0
            Linear-3                   [-1, 12]             156
              ReLU-4                   [-1, 12]               0
            Linear-5                   [-1, 12]             156
              ReLU-6                   [-1, 12]               0
            Linear-7                    [-1, 4]              52
            Linear-8                    [-1, 4]       

...

 88%|████████▊ | 7/8 [53:29<07:07, 427.91s/it]

R^2 score: 0.8572919920163176
R^2 Test score: 0.8375211051384788
{'config': {'x_dim': 10, 'z_dim': 4, 'zz_dim': 1, 'encoder_z_layers': [128, 128], 'encoder_zz_layers': [5, 5, 5], 'decoder_layers': [128, 128, 10], 'fc_z_to_y_layers': [28, 28, 1]}, 'beta': {'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 1000, 'b5': 1}, 'lr': 0.001, 'batch_size': 32, 'num_epochs': 1000}
False
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                  [-1, 128]           1,408
              ReLU-2                  [-1, 128]               0
            Linear-3                  [-1, 128]          16,512
              ReLU-4                  [-1, 128]               0
            Linear-5                    [-1, 4]             516
            Linear-6                    [-1, 4]             516
            Linear-7                    [-1, 5]              25
              ReLU-8                    [-1, 5]          

Epoch 201, loss: 499.75982666015625
Epoch 401, loss: 69.44424438476562
Epoch 601, loss: 49.134647369384766
Epoch 801, loss: 41.867820739746094
R^2 score: 0.7054186954971985
R^2 Test score: 0.8387009830191623
{'config': {'x_dim': 10, 'z_dim': 4, 'zz_dim': 1, 'encoder_z_layers': [128, 128], 'encoder_zz_layers': [5, 5, 5], 'decoder_layers': [128, 128, 10], 'fc_z_to_y_layers': [28, 28, 1]}, 'beta': {'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 1000, 'b5': 1}, 'lr': 0.001, 'batch_size': 32, 'num_epochs': 1000}
False
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                  [-1, 128]           1,408
              ReLU-2                  [-1, 128]               0
            Linear-3                  [-1, 128]          16,512
              ReLU-4                  [-1, 128]               0
            Linear-5                    [-1, 4]             516
            Linear-6                    [-

100%|██████████| 8/8 [58:58<00:00, 442.27s/it]

R^2 score: 0.8442574139747853
R^2 Test score: 0.8326898646324861


In [13]:
df3

,config,mean_r2,mean_r2_test,mean_loss_train,mean_loss_test
0,"{'config': {'x_dim': 10, 'z_dim': 4, 'zz_dim':...",0.702424,0.726823,0.076670,0.104577
1,"{'config': {'x_dim': 10, 'z_dim': 4, 'zz_dim':...",0.730589,0.714611,0.075437,0.100991
2,"{'config': {'x_dim': 10, 'z_dim': 4, 'zz_dim':...",0.684560,0.700093,0.091170,0.107429
3,"{'config': {'x_dim': 10, 'z_dim': 4, 'zz_dim':...",0.656245,0.701353,0.091029,0.111683
4,"{'config': {'x_dim': 10, 'z_dim': 4, 'zz_dim':...",0.805450,0.815924,0.045382,0.084325
5,"{'config': {'x_dim': 10, 'z_dim': 4, 'zz_dim':...",0.770813,0.819054,0.039108,0.093030
6,"{'config': {'x_dim': 10, 'z_dim': 4, 'zz_dim':...",0.805916,0.824318,0.047803,0.085863
7,"{'config': {'x_dim': 10, 'z_dim': 4, 'zz_dim':...",0.771362,0.811247,0.045683,0.086917


In [14]:
# Sort the DataFrame based on the "mean_r2" column in descending order and get the 5 largest values
best_entries = df3.nlargest(5, "mean_r2")

best_entries

,config,mean_r2,mean_r2_test,mean_loss_train,mean_loss_test
6,"{'config': {'x_dim': 10, 'z_dim': 4, 'zz_dim':...",0.805916,0.824318,0.047803,0.085863
4,"{'config': {'x_dim': 10, 'z_dim': 4, 'zz_dim':...",0.805450,0.815924,0.045382,0.084325
7,"{'config': {'x_dim': 10, 'z_dim': 4, 'zz_dim':...",0.771362,0.811247,0.045683,0.086917
5,"{'config': {'x_dim': 10, 'z_dim': 4, 'zz_dim':...",0.770813,0.819054,0.039108,0.093030
1,"{'config': {'x_dim': 10, 'z_dim': 4, 'zz_dim':...",0.730589,0.714611,0.075437,0.100991


In [19]:
df3.iloc[6].config

{'config': {'x_dim': 10,
  'z_dim': 4,
  'zz_dim': 1,
  'encoder_z_layers': [128, 128],
  'encoder_zz_layers': [5, 5, 5],
  'decoder_layers': [128, 128, 10],
  'fc_z_to_y_layers': [28, 28, 1]},
 'beta': {'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 1000, 'b5': 1},
 'lr': 0.001,
 'batch_size': 32,
 'num_epochs': 700}